In [1]:
import sys
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn import set_config; set_config(display='diagram')

sys.path.append("/Users/antonis/code/Ant-mel/legendary_game_recs/")
from preprocessing.preprocess_1_cleaning import *
from preprocessing.preprocess_2_features import *
from preprocessing.pipeline_the_unification import *
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler

from sklearn.pipeline import make_pipeline, FeatureUnion
from sklearn.pipeline import make_union
from sklearn.compose import make_column_transformer
import datetime
from sklearn.preprocessing import MultiLabelBinarizer

In [2]:
data = pd.read_csv('../raw_data/all_game_data_v1_corrected2')

In [3]:
piplined_data = pipeline_genre_ohe_only(data)

/Users/antonis/code/Ant-mel/legendary_game_recs/preprocessing/preprocess_1_cleaning.py:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column]=pd.to_datetime(df[column])
/Users/antonis/code/Ant-mel/legendary_game_recs/preprocessing/preprocess_2_features.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dupe['gen'] = None
/Users/antonis/code/Ant-mel/legendary_game_recs/preprocessing/preprocess_1_cleaning.py:161: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

In [4]:
piplined_data

,title,release_date,plays,playing,backlogs,wishlist,developers,avg_review,platforms,description,...,Simulator,Puzzle,Shooter,Strategy,Platform,Arcade,Sport,gen_1,gen_2,gen_3
0,Darling Duality: Winter Wish,2024-12-31,7,0,2,2,[Melancholy Marionette],2.2,"[Windows PC, Mac, Linux]",We all have multiple sides. Pick your poison! ...,...,0,0,0,0,0,0,0,0,0,1
1,Anger Foot,2024-12-31,80,1,82,297,"[Devolver Digital, Robbie Fraser]",3.3,[Windows PC],Anger Foot is a lightning-fast hard-bass blast...,...,0,0,1,0,0,0,0,0,0,1
2,Life Is Feudal: MMO,2023-12-31,9,0,5,1,[Long Tale Games],2.4,[Windows PC],Life is Feudal: MMO is an open-world survival ...,...,1,0,0,0,0,0,0,0,0,1
3,The Making of Karateka,2023-08-29,10,3,9,41,[Digital Eclipse],4.5,"[Windows PC, PlayStation 4, Xbox One, PlayStat...",Play the history. Go behind the scenes of Jord...,...,0,0,0,0,1,0,0,0,0,1
4,Enigma of Fear,2023-12-31,16,0,26,155,[Dumativa Game Studio],2.1,[Windows PC],"Become Mia, a paranormal detective searching f...",...,0,1,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36815,Final Crisis: Terrestrial Defense Police,1991-12-01,4,0,1,4,[Techno Grard],3.8,[PC-8801],"In the year 2991 AD, the Galactic Federal Gove...",...,0,0,1,0,0,0,0,1,0,0
36816,Level Up!,2009-11-25,6,0,0,1,[],2.7,[Web browser],"Level Up! is a Game created by Titch007uk, and...",...,0,0,0,0,0,0,0,0,1,0
36817,Sea Battle,2014-02-07,3,0,0,0,[BYRIL],2.2,"[Android, iOS]",Sea Battle is a board game everybody loves sin...,...,0,0,0,0,0,0,0,0,0,1
36818,We Are Illuminati,2017-11-02,18,0,0,0,[Tapps Games],2.0,"[Android, iOS]","Control minds, insert subliminal messages into...",...,1,0,0,1,0,0,0,0,0,1


In [28]:
num_transformer = Pipeline([('mm_scaler', MinMaxScaler())])

col_transformer = ColumnTransformer([('num_transformer', num_transformer,
                                   ['plays','playing','backlogs','wishlist','total_reviews','total_lists'])],
                                  remainder='passthrough')

def make_training_data(reference_data):
    dropped = drop_unnecesary_coulumns(reference_data)
    
    transformed = pd.DataFrame(col_transformer.fit_transform(dropped))
    
    X_train = transformed.drop(6, axis=1)
    y_train = transformed[6]
    
    return X_train, y_train

In [29]:
X_train, y_train = make_training_data(piplined_data)

In [30]:
X_train

,0,1,2,3,4,5,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.000130,0.000161,0.000167,0.000313,0.000263,0.000690,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.001717,0.000323,0.006833,0.046406,0.002105,0.017931,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.000174,0.000161,0.000417,0.000156,0.000000,0.000517,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.000196,0.000645,0.000750,0.006406,0.001316,0.002069,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,0.000326,0.000161,0.002167,0.024219,0.003158,0.004655,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36815,0.000065,0.000161,0.000083,0.000625,0.000000,0.000345,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
36816,0.000109,0.000161,0.000000,0.000156,0.000263,0.000345,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
36817,0.000043,0.000161,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
36818,0.000370,0.000161,0.000000,0.000000,0.000526,0.001207,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0


In [31]:
y_train

0        2.2
1        3.3
2        2.4
3        4.5
4        2.1
        ... 
36815    3.8
36816    2.7
36817    2.2
36818    2.0
36819    3.7
Name: 6, Length: 36820, dtype: float64